In [150]:
%matplotlib inline
# import some dependencies
import numpy as np
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set()
except ImportError:
    pass

import torch
from torch.autograd import Variable

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

#torch.manual_seed(101)

In [275]:
size = 10
x = np.random.randn(size)
data = Variable(torch.Tensor([x]))
data.shape, data[0]

(torch.Size([1, 10]), Variable containing:
 -0.8918
 -1.1745
  0.3639
  1.2711
  0.7261
 -0.0010
  0.0486
 -0.7838
  1.1956
  2.2585
 [torch.FloatTensor of size 10])

In [276]:
# def model():
#     mu = pyro.sample("mu", dist.normal, Variable(torch.zeros(size)), Variable(torch.ones(size)))
#     return pyro.sample("x", dist.normal, mu, Variable(torch.ones(size)))

def model():
    mu = pyro.param("mu", Variable(torch.randn(1), requires_grad=True))
    return pyro.sample("x", dist.normal, mu, Variable(torch.ones(1)))

def guide():
    return None
#     mu_param = pyro.param("mu_param", Variable(torch.randn(size), requires_grad=True))
#     return pyro.sample("mu", dist.delta, mu_param)

# use pyro.condition to constrain model sample site "x" to be equal to some fixed observed data
conditioned_model = pyro.condition(model, data={"x": data})

svi = pyro.infer.SVI(model=conditioned_model, guide=guide, optim=pyro.optim.SGD({"lr": 0.001}), loss="ELBO")

losses = []
for t in range(1000):
    # svi.step is called with whatever arguments model and guide take - in this case, nothing
    losses.append(svi.step())

# to get the value of mu, just call guide
mu_final = guide()

ValueError: The event size for the data and distribution parameters must match.
Expected x.size()[-1] == self.mu.size()[-1], but got 10 vs 1

In [271]:
model()

Variable containing:
-0.8357
[torch.FloatTensor of size 1]